# TASK #1: UNDERSTAND THE PROBLEM STATEMENT/GOAL



- This dataset contains weekly sales from 99 departments belonging to 45 different stores. 
- Our aim is to forecast weekly sales from a particular department.
- The objective of this case study is to forecast weekly retail store sales based on historical data.
- The data contains holidays and promotional markdowns offered by various stores and several departments throughout the year.
- Markdowns are crucial to promote sales especially before key events such as Super Bowl, Christmas and Thanksgiving. 
- Developing accurate model will enable make informed decisions and make recommendations to improve business processes in the future. 
- The data consists of three sheets: 
    - Stores
    - Features
    - Sales
- Data Source : https://www.kaggle.com/manjeetsingh/retaildataset

# TASK #2: IMPORT DATASET AND LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile


In [2]:
df = pd.read_csv('../data/data_processed.csv')
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,month,Type,Size
0,1,1,2010-05-02,24924.50,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,5,A,151315
1,1,2,2010-05-02,50605.27,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,5,A,151315
2,1,3,2010-05-02,13740.12,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,5,A,151315
3,1,4,2010-05-02,39954.04,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,5,A,151315
4,1,5,2010-05-02,32229.38,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,5,A,151315


In [3]:
# Drop the date
y = df['Weekly_Sales'] # target
X = df.drop(columns = ['Weekly_Sales', 'Date']) # features

In [4]:
X = pd.get_dummies(X, columns = ['Type', 'Store', 'Dept'], drop_first = True)

In [5]:
X.shape

(421570, 138)

In [6]:
y.shape

(421570,)

In [7]:
X.head()

,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,...,Dept_90,Dept_91,Dept_92,Dept_93,Dept_94,Dept_95,Dept_96,Dept_97,Dept_98,Dept_99
0,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,...,0,0,0,0,0,0,0,0,0,0
1,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,...,0,0,0,0,0,0,0,0,0,0
2,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,...,0,0,0,0,0,0,0,0,0,0
3,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,...,0,0,0,0,0,0,0,0,0,0
4,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# save columnames for later purposes
featurenames = list(X.columns)

In [9]:
X = np.array(X).astype('float32')
y = np.array(y).astype('float32')

In [10]:
# reshaping the array from (421570,) to (421570, 1)
y = y.reshape(-1,1)
y.shape

(421570, 1)

In [11]:
# spliting the data into train, validation and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.75)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5)

In [12]:
X_train.shape

(105392, 138)

In [13]:
X_val.shape

(158089, 138)

In [14]:
X_test.shape

(158089, 138)

# TRAIN XGBOOST USING SAGEMAKER Builtin Algorithm with Optimized Parameters

In [15]:
# Convert the array into dataframe in a way that target variable is set as the first column and followed by feature columns
# This is because sagemaker built-in algorithm expects the data in this format.

train_data = pd.DataFrame({'Target': y_train[:,0]})
for i in range(X_train.shape[1]):
    train_data[i] = X_train[:,i]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [16]:
train_data.head()

,Target,0,1,2,3,4,5,6,7,8,...,128,129,130,131,132,133,134,135,136,137
0,12629.080078,0.0,41.400002,3.016,0.000000,0.0,0.00,0.000000,0.000000,211.486465,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2252.310059,0.0,66.190002,2.732,0.000000,0.0,0.00,0.000000,0.000000,190.294830,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5703.939941,0.0,72.620003,2.780,0.000000,0.0,0.00,0.000000,0.000000,182.442413,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3924.139893,1.0,47.660000,3.162,0.000000,0.0,0.00,0.000000,0.000000,126.669266,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10619.610352,0.0,79.470001,3.452,6401.419922,0.0,1.01,744.859985,7363.359863,221.387741,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
val_data = pd.DataFrame({'Target':y_val[:,0]})
for i in range(X_val.shape[1]):
    val_data[i] = X_val[:,i]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
val_data.head()

,Target,0,1,2,3,4,5,6,7,8,...,128,129,130,131,132,133,134,135,136,137
0,5201.270020,0.0,86.910004,3.417,7365.259766,14.48,0.700000,3640.699951,2517.310059,222.538513,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15287.480469,0.0,67.790001,3.979,10665.280273,0.00,1095.849976,1053.959961,2647.449951,138.110199,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6520.379883,0.0,34.139999,2.983,0.000000,0.00,0.000000,0.000000,0.000000,211.117676,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,75.000000,0.0,48.299999,2.976,0.000000,0.00,0.000000,0.000000,0.000000,211.956024,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,13316.459961,0.0,69.160004,3.784,0.000000,0.00,0.000000,0.000000,0.000000,140.196991,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
val_data.shape

(158089, 139)

In [20]:
# save train_data and validation_data as csv files.
train_data.to_csv('../data/train.csv', header = False, index = False)
val_data.to_csv('../data/validation.csv', header = False, index = False)

In [21]:
# Boto3 is the Amazon Web Services (AWS) Software Development Kit (SDK) for Python
# Boto3 allows Python developer to write software that makes use of services like Amazon S3 and Amazon EC2

import sagemaker
import boto3
from sagemaker import Session

# Let's create a Sagemaker session
sagemaker_session = sagemaker.Session()
#bucket = Session().default_bucket() 
bucket = 'salesprediction-ml-sagemaker'
prefix = 'XGBoost-Regressor'
key = 'XGBoost-Regressor'
#Roles give learning and hosting access to the data
#This is specified while opening the sagemakers instance in "Create an IAM role"
role = sagemaker.get_execution_role()

In [22]:
print(role)

arn:aws:iam::103721820087:role/service-role/AmazonSageMaker-ExecutionRole-20190909T202771


In [23]:
# read the data from csv file and then upload the data to s3 bucket
import os
with open('../data/train.csv','rb') as f:
    # The following code uploads the data into S3 bucket to be accessed later for training
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(f)

# Let's print out the training data location in s3
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))

uploaded training data location: s3://salesprediction-ml-sagemaker/XGBoost-Regressor/train/XGBoost-Regressor


In [24]:
# read the data from csv file and then upload the data to s3 bucket

with open('../data/validation.csv','rb') as f:
    # The following code uploads the data into S3 bucket to be accessed later for training

    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation', key)).upload_fileobj(f)
# Let's print out the validation data location in s3
s3_validation_data = 's3://{}/{}/validation/{}'.format(bucket, prefix, key)
print('uploaded validation data location: {}'.format(s3_validation_data))

uploaded validation data location: s3://salesprediction-ml-sagemaker/XGBoost-Regressor/validation/XGBoost-Regressor


In [25]:
# creates output placeholder in S3 bucket to store the output
output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))

training artifacts will be uploaded to: s3://salesprediction-ml-sagemaker/XGBoost-Regressor/output


In [26]:
# This code is used to get the training container of sagemaker built-in algorithms
# all we have to do is to specify the name of the algorithm, that we want to use

# Let's obtain a reference to the XGBoost container image
# Note that all regression models are named estimators
# You don't have to specify (hardcode) the region, get_image_uri will get the current region name using boto3.Session

from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(boto3.Session().region_name, 
                          'xgboost',
                          #'0.90-2', 
                          '1.5-1') # Latest version of XGboost

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [27]:
# remember, the best hyperparameters found were:
# 'clf__colsample_bytree': 0.1, 'clf__max_depth': 4, 'clf__n_estimators': 25}

In [28]:
# Specify the type of instance that we would like to use for training 
# output path and sagemaker session into the Estimator. 
# We can also specify how many instances we would like to use for training

# Recall that XGBoost works by combining an ensemble of weak models to generate accurate/robust results. 
# The weak models are randomized to avoid overfitting

# num_round: The number of rounds to run the training.


# Alpha: L1 regularization term on weights. Increasing this value makes models more conservative.

# colsample_by_tree: fraction of features that will be used to train each tree.

# eta: Step size shrinkage used in updates to prevent overfitting. 
# After each boosting step, eta parameter shrinks the feature weights to make the boosting process more conservative.


Xgboost_regressor = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count = 1, 
                                       train_instance_type = 'ml.m5.2xlarge',
                                       output_path = output_location,
                                       sagemaker_session = sagemaker_session)

#We can tune the hyper-parameters to improve the performance of the model

Xgboost_regressor.set_hyperparameters(max_depth = 4,
                           objective = 'reg:linear',
                           colsample_bytree = 0.1,
                           # n_estimators = 25, --> not a valid hyperparameter for the built in version!
                           num_round = 100
                           )



train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [29]:
# Creating "train", "validation" channels to feed in the model
# Source: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html

train_input = sagemaker.session.s3_input(s3_data = s3_train_data, content_type='csv',s3_data_type = 'S3Prefix')
valid_input = sagemaker.session.s3_input(s3_data = s3_validation_data, content_type='csv',s3_data_type = 'S3Prefix')
data_channels = {'train': train_input,'validation': valid_input}
Xgboost_regressor.fit(data_channels)

The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2022-11-29 09:22:40 Starting - Starting the training job...
2022-11-29 09:23:04 Starting - Preparing the instances for trainingProfilerReport-1669713760: InProgress
.........
2022-11-29 09:24:24 Downloading - Downloading input data...
2022-11-29 09:25:05 Training - Downloading the training image..[2022-11-29 09:25:17.033 ip-10-0-142-184.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-11-29:09:25:17:INFO] Imported framework sagemaker_xgboost_container.training
[2022-11-29:09:25:17:INFO] Failed to parse hyperparameter objective value reg:linear to Json.
Returning the value itself
[2022-11-29:09:25:17:INFO] No GPUs detected (normal if no gpus installed)
[2022-11-29:09:25:17:INFO] Running XGBoost Sagemaker in algorithm mode
[2022-11-29:09:25:17:INFO] Determined delimiter of CSV input is ','
[2022-11-29:09:25:17:INFO] Determined delimiter of CSV input is ','
[2022-11-29:09:25:17:INFO] files path: /opt/ml/input/data/train
[2022-11-29:09:25:17:INFO] Determined delim

# TASK #11: DEPLOY THE MODEL TO MAKE PREDICTIONS

In [30]:
# Deploy the model to perform inference 
predictor = Xgboost_regressor.deploy(initial_instance_count = 1, instance_type = 'ml.m5.2xlarge')

-----!

In [33]:
'''
Content type over-rides the data that will be passed to the deployed model, since the deployed model expects data
in text/csv format, we specify this as content -type.

Serializer accepts a single argument, the input data, and returns a sequence of bytes in the specified content
type

Reference: https://sagemaker.readthedocs.io/en/stable/predictors.html
'''
from sagemaker.predictor import csv_serializer, json_deserializer

predictor.serializer = csv_serializer
predictor.deserializer = json_deserializer

In [34]:
X_test.shape

(158089, 138)

In [45]:
# making prediction
predictions1 = predictor.predict(X_test[0:10000])
predictions2 = predictor.predict(X_test[10000:20000])
predictions3 = predictor.predict(X_test[20000:30000])
predictions4 = predictor.predict(X_test[30000:40000])
predictions5 = predictor.predict(X_test[40000:50000])
predictions6 = predictor.predict(X_test[50000:60000])
predictions7 = predictor.predict(X_test[60000:70000])

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_seri

In [46]:
type(predictions2)

dict

In [47]:
# in case the json_deserializer has been used, then use this function for decoding the predictions into an array
def json_predictions_2_array(json_predictions:dict)->np.array:
    return np.array([pred['score'] for pred in json_predictions['predictions']])

In [48]:
predicted_values_1 = json_predictions_2_array(predictions1)
predicted_values_2 = json_predictions_2_array(predictions2)
predicted_values_3 = json_predictions_2_array(predictions3)
predicted_values_4 = json_predictions_2_array(predictions4)
predicted_values_5 = json_predictions_2_array(predictions5)
predicted_values_6 = json_predictions_2_array(predictions6)
predicted_values_7 = json_predictions_2_array(predictions7)

In [49]:
predicted_values = np.concatenate((predicted_values_1, 
                                   predicted_values_2,
                                  predicted_values_3,
                                  predicted_values_4,
                                  predicted_values_5,
                                  predicted_values_6,
                                  predicted_values_7))
predicted_values.shape

(70000,)

In [50]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from math import sqrt
k = X_test[:70000].shape[1]
n = len(X_test[:70000])
RMSE = float(format(np.sqrt(mean_squared_error(y_test[:70000], predicted_values)),'.3f'))
MSE = mean_squared_error(y_test[:70000], predicted_values)
MAE = mean_absolute_error(y_test[:70000], predicted_values)
r2 = r2_score(y_test[:70000], predicted_values)
adj_r2 = 1-(1-r2)*(n-1)/(n-k-1)

print('RMSE =',RMSE, '\nMSE =',MSE, '\nMAE =',MAE, '\nR2 =', r2, '\nAdjusted R2 =', adj_r2) 

RMSE = 11057.331 
MSE = 122264559.1669434 
MAE = 6523.909642425309 
R2 = 0.7605392853770345 
Adjusted R2 = 0.760066266401956


In [71]:
# in case the json_deserialzer has not been used, then the byte array can be decoded using this function
# custom code to convert the values in bytes format to array
def bytes_2_array(x):
    
    # makes entire prediction as string and splits based on ','
    l = str(x).split('n')[:-1]
    
    # Since the first element contains unwanted characters like (b,',') we remove them
    l[0] = l[0][2:]
    #same-thing as above remove the unwanted last character (')
    l[-1] = l[-1][:-1]
    
    # iterating through the list of strings and converting them into float type
    for i in range(len(l)):
        l[i] = float(l[i][:-1])
        
    # converting the list into array
    l = np.array(l).astype('float32')
    
    # reshape one-dimensional array to two-dimensional array
    return l.reshape(-1,1)
    

In [38]:
predicted_values_1 = bytes_2_array(predictions1)
predicted_values_1.shape

(10000, 1)

In [39]:
predicted_values_2 = bytes_2_array(predictions2)
predicted_values_2.shape

(10000, 1)

In [40]:
predicted_values_3 = bytes_2_array(predictions3)
predicted_values_3.shape

(10000, 1)

In [41]:
predicted_values_4 = bytes_2_array(predictions4)
predicted_values_4.shape

(1618, 1)

In [42]:
predicted_values = np.concatenate((predicted_values_1, predicted_values_2, predicted_values_3, predicted_values_4))

In [43]:
predicted_values.shape

(31618, 1)

In [44]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from math import sqrt
k = X_test.shape[1]
n = len(X_test)
RMSE = float(format(np.sqrt(mean_squared_error(y_test, predicted_values)),'.3f'))
MSE = mean_squared_error(y_test, predicted_values)
MAE = mean_absolute_error(y_test, predicted_values)
r2 = r2_score(y_test, predicted_values)
adj_r2 = 1-(1-r2)*(n-1)/(n-k-1)

print('RMSE =',RMSE, '\nMSE =',MSE, '\nMAE =',MAE, '\nR2 =', r2, '\nAdjusted R2 =', adj_r2) 

ValueError: Found input variables with inconsistent numbers of samples: [105392, 31618]

In [51]:
# Delete the end-point
predictor.delete_model()
predictor.delete_endpoint()